In [4]:
import pandas as pd
import numpy as np
from numpy import loadtxt
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from imblearn.over_sampling import SMOTE
from sklearn.metrics import confusion_matrix
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
import csv
from scipy import stats
# from sklearn.preprocessing import power_transform
from sklearn.preprocessing import PowerTransformer

from numpy import loadtxt

from sklearn.metrics import accuracy_score

from sklearn import metrics
import csv
from matplotlib import pyplot

from sklearn.linear_model import LinearRegression
from sklearn.datasets import load_boston
from sklearn.preprocessing import PolynomialFeatures
import warnings

from sklearn.metrics import roc_auc_score

import numpy as np
import pandas as pd
import datetime
import random

# Plots
import seaborn as sns
import matplotlib.pyplot as plt

# Models
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, AdaBoostRegressor, BaggingRegressor
from sklearn.kernel_ridge import KernelRidge
from sklearn.linear_model import Ridge, RidgeCV
from sklearn.linear_model import ElasticNet, ElasticNetCV
from sklearn.svm import SVR
from mlxtend.regressor import StackingCVRegressor
import lightgbm as lgb
from lightgbm import LGBMRegressor
from xgboost import XGBRegressor

# Stats
from scipy.stats import skew, norm
from scipy.special import boxcox1p
from scipy.stats import boxcox_normmax

# Misc
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import KFold, cross_val_score
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import scale
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import RobustScaler
from sklearn.decomposition import PCA

pd.set_option('display.max_columns', None)

# Ignore useless warnings
import warnings
warnings.filterwarnings(action="ignore")
pd.options.display.max_seq_items = 8000
pd.options.display.max_rows = 8000


In [17]:
# load data

# from imblearn.over_sampling import SMOTE, ADASYN
# from collections import Counter
# from sklearn.svm import LinearSVC

dataset = pd.read_csv('C:/Purdue/Courses/Fall/Data Mining/Project/mgmt571/bankruptcy_Train.csv')
dataset.drop_duplicates(keep = False, inplace = True)

#dataset[(np.abs(stats.zscore(dataset)) < 3).all(axis=1)]


# split data into train and test sets

seed = 7

# split data into X and y

X = dataset[dataset.columns[0:64]]
Y = pd.DataFrame(dataset[dataset.columns[64]])


# interaction = PolynomialFeatures(degree=2, include_bias=False, interaction_only=True)
# X = interaction.fit_transform(X)
# X = pd.DataFrame(X)

#X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=1234)




smote = SMOTE(ratio='minority')
X_sm, y_sm = smote.fit_resample(X,Y)

# X_sm_test, y_sm_test = smote.fit_resample(X_test,y_test)

X_train = pd.DataFrame(X_sm)
y_train = pd.DataFrame(y_sm)

# X_test = pd.DataFrame(X_sm_test)
# y_test = pd.DataFrame(y_sm_test)



# # Create correlation matrix
# corr_matrix = X.corr().abs()

# # Select upper triangle of correlation matrix
# upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(np.bool))

# # Find index of feature columns with correlation greater than 0.95
# to_drop = [column for column in upper.columns if any(upper[column] > 0.80)]

# # Drop features 
# X.drop(X[to_drop], axis=1, inplace = True)



# interaction = PolynomialFeatures(degree=2, include_bias=False, interaction_only=True)
# X = interaction.fit_transform(X)
# X = pd.DataFrame(X)

# pt = PowerTransformer()
# pt.fit(X)                       ## Fit the PT on training data
# X = pt.transform(X)## Then apply on all data
# X=pd.DataFrame(X)



#X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.1, random_state=1234)


In [18]:
#y = y_test
train_labels = y_sm
X = X_sm

In [19]:
kf = KFold(n_splits=12, random_state=42, shuffle=True)

# Define error metrics
def rmsle(y, y_pred):
    return np.sqrt(mean_squared_error(y, y_pred))

def cv_rmse(model, X= X):
    rmse = np.sqrt(-cross_val_score(model, X, train_labels, scoring="neg_mean_squared_error", cv=kf))
    return (rmse)

In [31]:
kf = KFold(n_splits=12, random_state=42, shuffle=True)

# Define error metrics
def rmsle(y, y_pred):
    return roc_auc_score(y, y_pred)

def cv_rmse(model, X= X):
    rmse = cross_val_score(model, X, train_labels, scoring="roc_auc", cv=kf)
    return (rmse)

In [32]:
# Light Gradient Boosting Regressor
lightgbm = LGBMRegressor(objective='regression', 
                       num_leaves=6,
                       learning_rate=0.01, 
                       n_estimators=7000,
                       max_bin=200, 
                       bagging_fraction=0.8,
                       bagging_freq=4, 
                       bagging_seed=8,
                       feature_fraction=0.2,
                       feature_fraction_seed=8,
                       min_sum_hessian_in_leaf = 11,
                       verbose=-1,
                       random_state=42)

# XGBoost Regressor
xgboost = XGBRegressor(learning_rate=0.01,
                       n_estimators=6000,
                       max_depth=4,
                       min_child_weight=0,
                       gamma=0.6,
                       subsample=0.7,
                       colsample_bytree=0.7,
                       objective='reg:linear',
                       nthread=-1,
                       scale_pos_weight=1,
                       seed=27,
                       reg_alpha=0.00006,
                       random_state=42)

# Ridge Regressor
ridge_alphas = [1e-15, 1e-10, 1e-8, 9e-4, 7e-4, 5e-4, 3e-4, 1e-4, 1e-3, 5e-2, 1e-2, 0.1, 0.3, 1, 3, 5, 10, 15, 18, 20, 30, 50, 75, 100]
ridge = make_pipeline(RobustScaler(), RidgeCV(alphas=ridge_alphas, cv=kf))

# Support Vector Regressor
svr = make_pipeline(RobustScaler(), SVR(C= 20, epsilon= 0.008, gamma=0.0003))

# Gradient Boosting Regressor
gbr = GradientBoostingRegressor(n_estimators=6000,
                                learning_rate=0.01,
                                max_depth=4,
                                max_features='sqrt',
                                min_samples_leaf=15,
                                min_samples_split=10,
                                loss='huber',
                                random_state=42)  

# Random Forest Regressor
rf = RandomForestRegressor(n_estimators=1200,
                          max_depth=15,
                          min_samples_split=5,
                          min_samples_leaf=5,
                          max_features=None,
                          oob_score=True,
                          random_state=42)

# Stack up all the models above, optimized using xgboost
stack_gen = StackingCVRegressor(regressors=(xgboost, lightgbm, svr, ridge, gbr, rf),
                                meta_regressor=xgboost,
                                use_features_in_secondary=True)

In [33]:
scores = {}

score = cv_rmse(lightgbm)
print("lightgbm: {:.4f} ({:.4f})".format(score.mean(), score.std()))
scores['lgb'] = (score.mean(), score.std())

lightgbm: 0.9997 (0.0002)


In [ ]:
score = cv_rmse(xgboost)
print("xgboost: {:.4f} ({:.4f})".format(score.mean(), score.std()))
scores['xgb'] = (score.mean(), score.std())

In [ ]:
score = cv_rmse(svr)
print("SVR: {:.4f} ({:.4f})".format(score.mean(), score.std()))
scores['svr'] = (score.mean(), score.std())

In [ ]:
score = cv_rmse(ridge)
print("ridge: {:.4f} ({:.4f})".format(score.mean(), score.std()))
scores['ridge'] = (score.mean(), score.std())

In [ ]:
score = cv_rmse(rf)
print("rf: {:.4f} ({:.4f})".format(score.mean(), score.std()))
scores['rf'] = (score.mean(), score.std())

In [ ]:
score = cv_rmse(gbr)
print("gbr: {:.4f} ({:.4f})".format(score.mean(), score.std()))
scores['gbr'] = (score.mean(), score.std())

In [ ]:
print('stack_gen')
stack_gen_model = stack_gen.fit(np.array(X), np.array(train_labels))

In [ ]:
print('lightgbm')
lgb_model_full_data = lightgbm.fit(X, train_labels)

In [ ]:
print('xgboost')
xgb_model_full_data = xgboost.fit(X, train_labels)

In [ ]:
print('Svr')
svr_model_full_data = svr.fit(X, train_labels)

In [ ]:
print('Ridge')
ridge_model_full_data = ridge.fit(X, train_labels)

In [ ]:
print('RandomForest')
rf_model_full_data = rf.fit(X, train_labels)

In [ ]:
print('GradientBoosting')
gbr_model_full_data = gbr.fit(X, train_labels)

In [ ]:
# Blend models in order to make the final predictions more robust to overfitting
def blended_predictions(X):
    return ((0.1 * ridge_model_full_data.predict(X)) + \
            (0.2 * svr_model_full_data.predict(X)) + \
            (0.1 * gbr_model_full_data.predict(X)) + \
            (0.1 * xgb_model_full_data.predict(X)) + \
            (0.1 * lgb_model_full_data.predict(X)) + \
            (0.05 * rf_model_full_data.predict(X)) + \
            (0.35 * stack_gen_model.predict(np.array(X))))

In [ ]:
# Get final precitions from the blended model
blended_score = rmsle(train_labels, blended_predictions(X))
scores['blended'] = (blended_score, 0)
print('RMSLE score on train data:')
print(blended_score)

In [ ]:
# Plot the predictions for each model
sns.set_style("white")
fig = plt.figure(figsize=(24, 12))

ax = sns.pointplot(x=list(scores.keys()), y=[score for score, _ in scores.values()], markers=['o'], linestyles=['-'])
for i, score in enumerate(scores.values()):
    ax.text(i, score[0] + 0.002, '{:.6f}'.format(score[0]), horizontalalignment='left', size='large', color='black', weight='semibold')

plt.ylabel('Score (RMSE)', size=20, labelpad=12.5)
plt.xlabel('Model', size=20, labelpad=12.5)
plt.tick_params(axis='x', labelsize=13.5)
plt.tick_params(axis='y', labelsize=12.5)

plt.title('Scores of Models', size=20)

plt.show()

In [5]:
# smote = SMOTE(ratio='minority')
# X_sm, y_sm = smote.fit_resample(X_train, y_train)
# fit model no training data

# model = RandomForestClassifier(n_estimators=10).fit(X_train, y_train)
#X_test = X_test.as_matrix()
# clf = XGBClassifier(max_depth= 6, learning_rate=0.1, n_estimators=1000,objective='binary:logistic',
#                       min_child_weight = 8, feature_selector = 'greedy',reg_alpha= 0.005,
#                       gamma=1, subsample=0.6, colsample_bytree=0.8)

clf = XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0,
              learning_rate=0.01, max_delta_step=0, max_depth=3,
              min_child_weight=1, missing=None, n_estimators=100, n_jobs=1,
              nthread=None, objective='binary:logistic', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, subsample=1, verbosity=1)

#clf = XGBClassifier()

eval_set  = [(X_train, y_train), (X_test,y_test)]
#clf.fit(X_sm, y_sm, eval_set=eval_set, eval_metric="auc", early_stopping_rounds=30)

#clf.fit(X_train, y_train, eval_set=eval_set, eval_metric="auc" , early_stopping_rounds=30)

clf.fit(X_train, y_train, eval_set=eval_set, eval_metric="auc" )

C:\Users\rolig\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:219: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\rolig\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:252: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


[0]	validation_0-auc:0.864454	validation_1-auc:0.847507
[1]	validation_0-auc:0.870859	validation_1-auc:0.856278
[2]	validation_0-auc:0.875166	validation_1-auc:0.85737
[3]	validation_0-auc:0.887984	validation_1-auc:0.870018
[4]	validation_0-auc:0.887723	validation_1-auc:0.869178
[5]	validation_0-auc:0.88862	validation_1-auc:0.872159
[6]	validation_0-auc:0.905983	validation_1-auc:0.889811
[7]	validation_0-auc:0.905692	validation_1-auc:0.888846
[8]	validation_0-auc:0.908043	validation_1-auc:0.89238
[9]	validation_0-auc:0.909052	validation_1-auc:0.892349
[10]	validation_0-auc:0.909826	validation_1-auc:0.892111
[11]	validation_0-auc:0.910062	validation_1-auc:0.892981
[12]	validation_0-auc:0.913413	validation_1-auc:0.897761
[13]	validation_0-auc:0.914047	validation_1-auc:0.897877
[14]	validation_0-auc:0.914665	validation_1-auc:0.899166
[15]	validation_0-auc:0.915476	validation_1-auc:0.899468
[16]	validation_0-auc:0.915925	validation_1-auc:0.899009
[17]	validation_0-auc:0.915619	validation_1-

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0,
              learning_rate=0.01, max_delta_step=0, max_depth=3,
              min_child_weight=1, missing=None, n_estimators=100, n_jobs=1,
              nthread=None, objective='binary:logistic', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, subsample=1, verbosity=1)

In [6]:
y_pred = clf.predict(X_test)
predictions = [round(value) for value in y_pred]


conf_mat = confusion_matrix(y_true=y_test, y_pred=y_pred)
print('Confusion matrix:\n', conf_mat)

# evaluate predictions

accuracy = accuracy_score(y_test, predictions)
print("Accuracy: %.2f%%" % (accuracy * 100.0),'\n')

print (roc_auc_score(y_test, predictions))

Confusion matrix:
 [[738 229]
 [ 99 868]]
Accuracy: 83.04% 

0.8304033092037229


In [ ]:
# Load test dataset
df_test = pd.read_csv('C:/Purdue/Courses/Fall/Data Mining/Project/mgmt571/bankruptcy_Test_X.csv')
df_test=df_test.drop(['ID'],axis=1)
# features = ['Attr1', 'Attr2', 'Attr3', 'Attr4', 'Attr5', 'Attr6', 'Attr8', 'Attr9',
#        'Attr12', 'Attr13', 'Attr15', 'Attr16', 'Attr19', 'Attr20', 'Attr21',
#        'Attr24', 'Attr27', 'Attr28', 'Attr29', 'Attr30', 'Attr32', 'Attr33',
#        'Attr34', 'Attr36', 'Attr37', 'Attr38', 'Attr41', 'Attr45', 'Attr48',
#        'Attr50', 'Attr53', 'Attr55', 'Attr57', 'Attr61']

# df_test = df_test[features]
df_test = df_test.as_matrix()
pred = clf.predict_proba(df_test)
test = pd.DataFrame (pred)

test[1].to_csv('C:/Purdue/Courses/Fall/Data Mining/Project/mgmt571/result_r_latest.csv')